In [1]:
import sqlite3 as lite
import datetime as dt
import sys

In [11]:
# %load 'db_management.py'
import sqlite3 as lite
import sys


def create_table(debug=False):
    # Create DHT_data table if it doesn't exist already
    con = lite.connect('sensorsData.db')
    with con:
        cur = con.cursor()
        #if table 'DHT_data' exists:
        try:
            cur.execute("CREATE TABLE DHT_data(timestamp DATETIME, temp NUMERIC, hum NUMERIC)")
            if debug:
                print("Created DHT_data table")
        except:
            if debug:
                print("DHT_data table already exists")


def add_entry(timestamp, temp, hum, debug=False):
    # Ensure the 'sensorsData.db' exists:
    create_table()
    # Add a temp/humidity reading to the database
    con = lite.connect('sensorsData.db')
    with con:
        cur = con.cursor()
        if debug:
            print("Adding DHT reading to DHT_data table: ", timestamp, temp, hum)
        cur.execute("INSERT INTO DHT_data VALUES((?), (?), (?))", (timestamp, temp, hum))




In [12]:
# %load '../helpers.py'
# helper functions

import adafruit_dht
import board
import time
import psutil

def init_dht(model=22):
    model = int(model)
    print(model)
    flag = True
    while flag:
        try:
            print("attempting to initialize device...")
            if model == int(22):
                print("DHT22...")
                dhtDevice = adafruit_dht.DHT22(board.D4)
            elif model == int(11):
                print("DHT11...")
                dhtDevice = adafruit_dht.DHT11(board.D4)
            else:
                print("incorrect DHT model number--enter '11' or '22'")
                return None
            dhtDevice.temperature
            print("successfully initialized DHT" + str(model))

        except RuntimeError as error:
            print(error.args[0])
            print("failed to initialize, cleaning up libgpio processes...")
            for p in psutil.process_iter():
                if p.name()[:8] == 'libgpiod':
                    print("killing {} (pid: {})".format(p.name(), p.pid))
                    p.kill()
            time.sleep(5)
            continue
        flag = False
    return dhtDevice


def read_dht(dhtDevice, deg='F'):
    while True:
        try:
            t = dhtDevice.temperature
            h = dhtDevice.humidity
            if deg == 'F':
                t = t * (9 / 5) + 32
            return (t, h)
        except RuntimeError as error:
            print(error.args[0])
            time.sleep(2)




In [19]:
create_table(debug=True)

Created DHT_data table


In [14]:
DHT = init_dht(22)

22
attempting to initialize device...
DHT22...
A full buffer was not returned. Try again.
failed to initialize, cleaning up libgpio processes...
killing libgpiod_pulsein (pid: 1381)
attempting to initialize device...
DHT22...
A full buffer was not returned. Try again.
failed to initialize, cleaning up libgpio processes...
killing libgpiod_pulsei (pid: 1381)
killing libgpiod_pulsein (pid: 1383)
attempting to initialize device...
DHT22...
successfully initialized DHT22


In [18]:
t, h = read_dht(DHT)
now = dt.datetime.now()
add_entry(now, t, h, debug=True)

Adding DHT reading to DHT_data table:  2020-12-11 22:58:38.836630 63.14 62.0


In [37]:
import sqlite3 as lite
import sys

def create_table(location, debug=False):
    # Create DHT_data table if it doesn't exist already
    con = lite.connect(os.path.join(location, 'sensorsData.db'))
    with con:
        cur = con.cursor()
        #if table 'DHT_data' exists:
        try:
            cur.execute("CREATE TABLE DHT_data(timestamp DATETIME, temp NUMERIC, hum NUMERIC)")
            if debug:
                print("Created DHT_data table")
        except:
            if debug:
                print("DHT_data table already exists")
    return con


def add_entry(db, timestamp, temp, hum, debug=False):
    # Add a temp/humidity reading to the database
    with db as con:
        cur = con.cursor()
        if debug:
            print("Adding DHT reading to DHT_data table: ", timestamp, temp, hum)
        cur.execute("INSERT INTO DHT_data VALUES((?), (?), (?))", (timestamp, temp, hum))

In [39]:
db = create_table('../db', debug=True)

DHT_data table already exists


In [41]:
add_entry(db, dt.datetime.now(), t, h, debug=True)

Adding DHT reading to DHT_data table:  2020-12-11 23:38:45.068689 63.14 62.0


In [35]:
os.path.isdir('../db')

True

In [34]:
os.mkdir('../db')